In [1]:
import pandas as pd
import numpy as np
import gudhi as gd
from gudhi.point_cloud.timedelay import TimeDelayEmbedding
from gudhi.hera import wasserstein_distance
from gudhi.representations import PersistenceImage
from gudhi.representations.metrics import WassersteinDistance
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.manifold import MDS
import os
%matplotlib widget

In [2]:
datapath =".\\data\\timeseries\\good"
filelist = os.listdir(datapath)
print(sorted(filelist))
good_time_series = [np.array(pd.read_csv(os.path.join(datapath,f), header=None)[0]) for f in sorted(filelist)]
datapath =".\\data\\timeseries\\bad"
filelist = os.listdir(datapath)
print(sorted(filelist))
bad_time_series = [np.array(pd.read_csv(os.path.join(datapath,f), header=None)[0]) for f in sorted(filelist)]

['s1_dobry.csv', 's2_dobry.txt', 's3_dobry.txt', 's4_dobry.txt', 's5_dobry.txt']
['s1_zly.csv', 's2_zly.txt', 's3_zly.txt', 's4_zly.txt', 's5_zly.txt']


In [ ]:
dim = 4
delay =2
skip = 1

tde = TimeDelayEmbedding(dim = dim, delay=delay, skip=skip)
good_point_clouds = tde.transform(good_time_series)
bad_point_clouds = tde.transform(bad_time_series)

print(np.array(good_point_clouds).shape)

In [3]:
good_pds = []

for pc in good_point_clouds:
    ac = gd.AlphaComplex(pc)
    st = ac.create_simplex_tree()
    st.compute_persistence()
    good_pds.append(st.persistence_intervals_in_dimension(1))

bad_pds = []

for pc in bad_point_clouds:
    ac = gd.AlphaComplex(pc)
    st = ac.create_simplex_tree()
    st.compute_persistence()
    bad_pds.append(st.persistence_intervals_in_dimension(1))

NameError: name 'good_point_clouds' is not defined

In [ ]:
f,axes = plt.subplots(1,len(good_pds), figsize = (20,4))
for i in range(0,len(good_pds)):
    gd.plot_persistence_diagram(good_pds[i], axes=axes[i])
    gd.plot_persistence_diagram(bad_pds[i], axes=axes[i],colormap=cm.Set2.colors)
plt.show()

In [ ]:
pi = PersistenceImage()
pi.fit(good_pds+bad_pds)
good_pis = pi.transform(good_pds)
bad_pis = pi.transform(bad_pds)

In [ ]:
pis = np.concatenate((good_pis,bad_pis))
mds = MDS(n_components=2)
emb =mds.fit_transform(pis)

In [ ]:
f,ax = plt.subplots()
colors = ["tab:blue","tab:orange", "tab:green", "tab:red","tab:olive"]
ax.scatter(emb[:5,0],emb[:5,1], marker = "o", c= colors)
ax.scatter(emb[5:,0],emb[5:,1], marker = "x", c=colors)
plt.show()

In [ ]:
len(good_pds+bad_pds)